II-34-2 Feynamn Lectures on Physics

https://www.feynmanlectures.caltech.edu/II_34.html

 mu = q*v*r/2


In [1]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi
using SymbolicUtils # for symbolic simplification

functions for generating monomials

In [5]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

all_monomials_up_to_max_deg_overall_and_individual (generic function with 1 method)

In [2]:
@polyvar A pi r mu I qc freq v   # 

x   =   [ A pi r mu I qc freq v  ] #

1×8 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 A  pi  r  mu  I  qc  freq  v

In [3]:
axioms= 
[
    A - pi * r^2,               # A = pi * r^2
    mu - I * A,                 # mu = I * A
    I - qc * freq,               # I = q * freq
    freq * (2 * pi * r) - v,    # freq = v/(2 * pi * r)
]


4-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 A - pir²
 mu - AI
 I - qcfreq
 -v + 2pirfreq

In [6]:
deg = 3
deg_overall = 10
theDegrees = [ 2  2 3  2 2  2  2   2 ]   
            #  A pi r mu I qc freq v 

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

#model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([ mu qc v r ], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [mu]).>0]) == 1.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(6924,), (6924,), (6924,), (6924,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
Set parameter TimeLimit to value 600
size.(coeff_αs) = [(6924,), (6924,), (6924,), (6924,)]
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 26839 rows, 27947 columns and 55830 nonzeros
Model fingerprint: 0x8d978fcb
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 26839 rows and 27947 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   

OPTIMAL::TerminationStatusCode = 1

In [7]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#31 (generic function with 1 method)

In [8]:
value_q = value_poly(q)
value_q

mu - 0.5rqcv

In [9]:
q_sim = simplify(value_q)

mu - 0.5rqcv

which is indeed the knwon relation mu = q r v / 2

In [10]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[qcfreq, 1.0, A, 0.5rqc]


4-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 qcfreq
 1.0
 A
 0.5rqc